In [1]:
#import wikitextparser as wtp
import requests

from codex.database.vasp import generate_database, refresh_database, tidy_wikicode
USER_AGENT = "dft-codex/0.0.0 (Omar A. Ashour, ashour@berkeley.edu)"
API_URL = "https://www.vasp.at/wiki/api.php"
REST_API_URL = "https://www.vasp.at/wiki/rest.php/v1/"

import logging 
import re
logging.basicConfig(level=logging.INFO)
import html
from copy import deepcopy
from datetime import timezone, datetime

from lxml.etree import tostring
from lxml.html import parse, fromstring, tostring

from importlib import resources
import os
import json

from codex.database.vasp import parse_incar_tag, get_from_vasp_wiki, tidy_wikicode, _style_tag_values

In [2]:
database = generate_database()
#########################
#database_dir = "vasp-1686736265"  # TODO: this is temporary
#base_db_dir = "codex/database"
#db_filename = os.path.join(base_db_dir, database_dir, f"database.json")
#database_without_wikicode = deepcopy(database)
#for category, pages in database_without_wikicode.items():
#    for title, page in pages.items():
#            del page["wikicode"]
#with open(db_filename, "w") as f:
#    json.dump(database_without_wikicode, f)

Removing header and footer from page: PREC
Removing header and footer from page: ICHARG
Removing header and footer from page: ISTART
Removing header and footer from page: INIWAV
Removing header and footer from page: NSW
Removing header and footer from page: NELM
Removing header and footer from page: EDIFF
Removing header and footer from page: ALGO
Removing header and footer from page: LMAXMIX
Removing header and footer from page: LVTOT
Removing header and footer from page: NELMDL
Removing header and footer from page: NELMIN
Removing header and footer from page: IALGO
Removing header and footer from page: NBANDS
Removing header and footer from page: NELECT
Removing header and footer from page: LREAL
Removing header and footer from page: ROPT
Removing header and footer from page: AEXX
Removing header and footer from page: AGGAX
Removing header and footer from page: AGGAC
Removing header and footer from page: ALDAC
Removing header and footer from page: LHFCALC
Removing header and footer f

AttributeError: 'NoneType' object has no attribute 'remove'

In [ ]:
print(database['INCAR']['MAXMIX']['info'])

In [3]:
base_db_dir = "codex/database"
html_cache_path = os.path.join(base_db_dir, "vasp-1686736265", "incar-raw-html.json")
with open(html_cache_path, "r") as f:
    html_cache = json.load(f)

In [20]:
tag_name = "IBRION"
tag_html = html_cache.get(tag_name, None)
page = get_from_vasp_wiki(tag_name)[0]
#name, tag = parse_incar_tag(page, tag_html)

In [23]:
root = fromstring(tag_html)
header = root.xpath("//*[starts-with(text(), 'Description:')]")
for element in header:
    preceding_siblings = element.xpath("preceding-sibling::*")
    for sibling in preceding_siblings:
        sibling.getparent().remove(sibling)
    element.getparent().remove(element)
footer = root.xpath("//*[starts-with(text(), 'Related Tags')]")
for element in footer:
    parent = element.getparent()
    preceding_siblings = element.xpath("following-sibling::*")
    for sibling in preceding_siblings:
        sibling.getparent().remove(sibling)
    parent.getparent().remove(parent)
references = root.xpath("//*[starts-with(text(), 'References')]")
new_html = tostring(root, encoding="unicode")
print("*********=============*******")
print(new_html)

*********=============*******
<div class="mw-parser-output"><hr>
<p>For <a class="mw-selflink selflink">IBRION</a>=0, a molecular dynamics is performed, whereas all other algorithms are destined for relaxations into a local energy minimum. For difficult relaxation problems it is recommended to use the conjugate gradient algorithm (<a class="mw-selflink selflink">IBRION</a>=2), which presently possesses the most reliable backup routines. Damped molecular dynamics (<a class="mw-selflink selflink">IBRION</a>=3) are often useful when starting from very bad initial guesses. Close to the local minimum the RMM-DIIS (<a class="mw-selflink selflink">IBRION</a>=1) is usually the best choice. <a class="mw-selflink selflink">IBRION</a>=5 and <a class="mw-selflink selflink">IBRION</a>=6 are using finite differences to determine the second derivatives (Hessian matrix and phonon frequencies), whereas <a class="mw-selflink selflink">IBRION</a>=7 and <a class="mw-selflink selflink">IBRION</a>=8 use den

In [ ]:
tag_html = html
tag_html.rsplit("<hr/>", 1)[0].split("<hr/>", 1)[-1]

In [ ]:
print(tag_html.replace('<hr />', '****************'))

In [ ]:
wikicode = page["text"]
sections = wikicode.get_sections()
for section in sections:
    print(section)
    print('************')
#lambda x: x.title.starswith("Related"))
#(lambda x: x.title.starswith("Related"))

In [ ]:
print(page['text'])

In [ ]:
page_html = tag['info']
page_html = re.sub(r'ICHARG<\/a>\+10', 'ICHARG</a>=10', page_html)
print(page_html)

In [ ]:
from html import unescape
unescape('&#8805;')

In [ ]:
wikicode = "for {{TAG|IBRION}}{{=}}0 (molecular dynamics) or {{TAG|LHFCALC}}{{=}}.TRUE."
wikicode_html = tidy_wikicode(wikicode, style_tag_values=False)
el = fromstring(wikicode_html)
for a in el.xpath("//a[@class='tag-link']"):
    _style_tag_values(a)

In [ ]:
tostring(el).decode("utf-8")

In [ ]:
for tag, value in database['INCAR'].items():
    print(f'{tag}')# {tagdef_templates[tag]} <---> {def_templates[tag]}')
    if value['datatype'] != "Unknown":
        print(f"*** datatype: {value['datatype']}")
    print(f"*** summary: {value['summary']}")
    if value['options']:
        print(f"*** options: {value['options']}")
    if value['default']:
        print(f"*** default: {value['default']}")
    print(f"-----------------------------------")

In [ ]:
import json
with open('codex/database/vasp-1686736265/database.json', 'r') as f:
    database = json.load(f)

In [ ]:
database['INCAR']['LDAU'].keys()

# HTML conversion (mwcomposerfromhell)

# Workflow
We want to parse 5 specific pages + every INCAR tag.

#### Option 1: Database exists
1. pull *only* the time stamp of the 5 pages + every INCAR tag.
2. compare time stamps to those in the database
3. update list of pages to parse, only need those that have been updated
4. parse the list of pages

#### Option 2: Database doesn't exist
1. pull the default list of pages 


In [ ]:
fudge_dates = True
# Change the date on 20 random entries in the database to 1980
import random
if fudge_dates and database:
    for i in range(30):
        title = random.choice(list(database['INCAR'].keys()))
        database['INCAR'][title]["last_revised"] = "1980-01-01T00:00:00Z"
        #print(f"Changed ['INCAR']['{title}'] to 1980-01-01T00:00:00Z")

In [ ]:
if not database:
    logging.info("Database doesn't exist. Creating it now.")
    database = generate_database()
    logging.info("Database created.")
else:
    logging.info("Database exists. Checking for updates.")
    database = refresh_database(database)

# Defaults, Options and Data Type

These can be obtained from `TAGDEF` and maybe `DEF` templates, depending on your luck...

The format is:

```
{{TAGDEF | TAGNAME | DATATYPE | DEFAULT}}
```

where DATATYPE is either:
```
1. integer
2. real
3. logical (also written as .TRUE. {{!}} .FALSE.)
4. (something) array (only seen real from a cursory look)
5. A list of options (e.g., 1 {{!}} 2-10 {{!}} [integer] > 0)
```

DEFAULT isn't always there, and if it is, could be a string with some more explanation or even 'not set' or '<math>10^{-4}</math>' (with the HTML tags).

the DEF template has the form:
```
{{DEF | TAGNAME | VALUE | CONDITION | VALUE | CONDITION | ....}}
```
where CONDITION is a string describing other tags (usually) and VALUE is an integer or string or whatever

lam

Example:
```
{{TAGDEF|PREC|Low {{!}} Medium {{!}} High {{!}} Normal {{!}} Single {{!}} Accurate}}
```

(no default, list of values)

```
{{DEF|ISTART|1|if a {{FILE|WAVECAR}} file exists|0|else}}
```


# STRETCH GOAL
The formatting of these things is utter shit, it might be really diffcult to do the comment to explain value thing...

Look at the difference between ISMEAR and ICHARG. They're in lists, with possible indented list subitems, descriptions on same line or not, typos meaning there's no '=' after the value....

Focus on the HTML webpage for now

In [ ]:
tag = "ICHARG"
text = database[tag]["text"]
sections = text.get_sections()
options = []
#for section in sections:
#    if section.title is not None:
#        # Strip title of whitespace (leading and trailing)
#        if section.title.strip() == "Tag options":
#            lists = section.get_lists()
wikilink="*{{TAG|"+tag+"}}"
lists = text.get_lists()
for lll in lists:
    for item in lll.fullitems:
        if item.startswith(wikilink):
            options.append(item.split(wikilink)[1].strip())

print(options)
#options_dict = {}
#for option in options:
#    setting = wtp.parse(option.split(":")[0]).plain_text()
#    explanation = wtp.parse(option.split(":")[1]).plain_text()
#    options_dict.update({setting: explanation})

#options_dict

# PAW stuff

In [ ]:
tag = "Available PAW potentials"
text = database[tag]["text"]
sections = text.get_sections()[1:3]
bolds = []
for section in sections:
    print(section.title)
    for b in section.get_bolds():
        try:
            float(b.plain_text())
        except:
            print("Found recommended pseudo: " + b.plain_text())
            # TODO: doesn't work for the GW ones....
            bolds.append(b.plain_text())

#bolds